# Data Transformation and Manipulation

In [2]:
# Reading parquet file

df = spark.read.format('parquet')\
                .load('abfss://refined@projectsynapsestorage.dfs.core.windows.net/NoNulls/*.parquet')

In [3]:
display(df)

# withColumn()

In [5]:
# Creating new Column named UnEmployed Rate Percentage
# (Unemployed/Labor Force )*100 =  UnEmployed Rate Percentage

from pyspark.sql.functions import col 
df_newcol = df.withColumn('UnEmployed Rate Percentage', (col('Unemployed') / col('Labor Force')) *100)

In [6]:
display(df_newcol)

# Distinct Function

In [7]:
# Understanding distinct values in the Education level column

df_newcol.select(col('Education Level')).distinct().show()

### With Column to update the column

In [8]:
# withColumn - Update

from pyspark.sql.functions import initcap

df_cap = df_newcol.withColumn('Education Level', initcap('Education Level'))

In [10]:
display(df_cap)

In [11]:
# Understanding distinct values in the Education level column

df_cap.select(col('Education Level')).distinct().show()

In [13]:
# Usign WithColumn to change the Some College to College
from pyspark.sql.functions import when

df_col = df_cap.withColumn('Education Level',when(col('Education Level') == 'Some College','College').otherwise(col('Education Level')))

In [14]:
display(df_col)

In [15]:
# Understanding distinct values in the Education level column

df_col.select(col('Education Level')).distinct().show()

In [22]:
df_col.withColumn('Education Level',\
                            when(col('Education Level') == "Bachelor's","Bachelor's Degree")\
                              .when(col('Education Level') == "Master's","Master's Degree")\
                              .when(col('Education Level') == "Associate's","Associate's Degree")\
                              .when(col('Education Level') == 'Associate',"Associate's Degree")\
                              .otherwise(col('Education Level'))).select(col('Education Level')).distinct().show()


In [24]:
df_changed = df_col.withColumn('Education Level',\
                              when(col('Education Level') == '''Bachelor's''','''Bachelor's Degree''' )\
                              .when(col('Education Level') == '''Master's''','''Master's Degree''')\
                              .when(col('Education Level') == '''Associate's''','''Associate's Degree''')\
                              .when(col('Education Level') == 'Associate','''Associate's Degree''')\
                              .otherwise(col('Education Level')))

In [25]:
display(df_changed)

# drop() function

In [26]:
df_changed = df_changed.drop('Unemployment Rate')

In [27]:
display(df_changed)

# withColumnRenamed() function

In [30]:
df_new = df_changed.withColumnRenamed(existing='Line Number',new='Line_Number')

In [31]:
display(df_new)

In [32]:
df_new.write.format('parquet')\
            .mode('overwrite')\
            .save('abfss://refined@projectsynapsestorage.dfs.core.windows.net/DataTransformed/')